# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [1]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/7

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
!pip install -qU qdrant-client ragas python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6

We'll also provide our OpenAI key, as well as our Cohere API key.

In [163]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key: ········


In [165]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key: ········


In [169]:
os.environ["CO_API_KEY"] = os.environ["COHERE_API_KEY"]

In [1]:
from dotenv import load_dotenv; _ = load_dotenv(override=True)

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [253]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2024-09-29 16:52:50--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.001s  

2024-09-29 16:52:51 (18.2 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2024-09-29 16:52:51--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [5]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 26, 17, 0, 20, 68160)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [6]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [7]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [10]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [63]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The movie was praised for its slickness, brilliant action sequences, and Keanu Reeves' performance as the titular character. Reviewers mentioned that it was a cool, fun, and violent film that kept viewers entertained and wanting more."

In [64]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review Title: A Masterpiece & Brilliant Sequel\n- Review URL: '/review/rw4854296/?ref_=tt_urv'"

In [65]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hit-man comes out of retirement to track down the gangsters that killed his dog and took everything from him. With the untimely death of his beloved wife still bitter in his mouth, he seeks vengeance. The story follows John Wick as he unleashes a carefully orchestrated maelstrom of destruction against those who attempt to chase him. He becomes the target of hitmen and faces an army of bounty-hunting killers on his trail. The legendary hitman is forced to unearth his concealed identity and carry out a relentless vendetta. Ultimately, only blood can quench his thirst for retribution.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [17]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [18]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [19]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, opinions on John Wick seem to vary. Some people really enjoyed the action and style of the movie, while others found it lacking in plot and substance. It seems like there are mixed feelings about the John Wick movies, with some loving them and others not being impressed.'

In [20]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I don't know if any reviews have a rating of 10."

In [21]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is an action movie series that features beautifully choreographed action scenes, emotional setups, and a strong performance by Keanu Reeves. It has been praised for its intense action and engaging storyline.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [22]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [23]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided in the context.'

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [26]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, John Wick, is forced back into his old life as a hitman after a mobster blows up his house and asks for his help. Wick is given a mission to kill Santino's sister, but when he completes it, Santino puts a bounty on his head. Wick then seeks revenge and promises to kill Santino."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [28]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [29]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The film received high ratings and praise for its action sequences, Keanu Reeves' performance, and overall entertainment value."

In [30]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?\n\nYes, there is a review with a rating of 10. Here is the URL to that review:\n- https://www.imdb.com/review/rw4854296/?ref_=tt_urv'

In [31]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, portrayed by Keanu Reeves, is a retired assassin who comes out of retirement when someone kills his dog. This sets off a chain of events where John Wick seeks revenge against those who have wronged him, leading to a lot of action-packed sequences where he takes on various adversaries. The story involves a lot of violence and intense action as John Wick navigates through the criminal underworld.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [32]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [33]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/_s/6j2r22wj52g7wsz019z7jgsm0000gn/T/ipykernel_44579/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [34]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [35]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [36]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [40]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, opinions about John Wick seem to vary. Some people love the series, while others did not enjoy the latest installment, John Wick 4. So, it's safe to say that not everyone generally liked John Wick."

In [41]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [42]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, played by Keanu Reeves, is a retired assassin who comes out of retirement after his dog is killed and his car is stolen. He is forced back into the world of assassins when he is called on to pay off an old debt by helping take over the Assassin's Guild. This leads to a lot of carnage and action as he travels to Italy, Canada, and Manhattan to carry out his missions."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [43]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [44]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [45]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, it seems that the majority of people generally liked John Wick. There were multiple positive reviews praising the action sequences, Keanu Reeves' performance, and the overall entertainment value of the movie. However, there were also some negative reviews that criticized certain aspects of the film. Overall, it can be concluded that John Wick was well-received by a significant portion of the audience."

In [46]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- [Review: A Masterpiece & Brilliant Sequel](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [47]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hitman comes out of retirement to track down the gangsters that killed his dog and took everything from him. With his wife\'s death still fresh, he seeks vengeance and unleashes a maelstrom of destruction against those who try to chase him. The movie is filled with action, shootouts, and breathtaking fights as John Wick seeks retribution for the wrongs done to him.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [48]:
!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [49]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [50]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [51]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [52]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [53]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [54]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [55]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\''

In [56]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, John Wick, seeks revenge on the people who took something he loved from him. Specifically, they killed his dog and stole his car, leading him to go on a path of vengeance against the Russian gangsters responsible.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

## Testset Generation
Let's first generate a testset making use of RAGAS and more powerful LLMs.

In [ ]:
!pip install -qU ipywidgets

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [22]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pandas as pd

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.7,
    multi_context: 0.2,
    reasoning: 0.1
}

num_qa_pairs = 100

In [ ]:
# We'll use parent_docs to generate questions. Cleared cell outputs to avoid noise in the notebook
testset = generator.generate_with_langchain_docs(documents, num_qa_pairs, distributions)

In [26]:
df = testset.to_pandas()
df.to_csv('saved_data/golden_eval_set.csv')

In [27]:
import pickle
with open('saved_data/testset.bin', 'wb') as f:
  pickle.dump(testset, f)

In [212]:
# Read back generated testset
import pandas as pd
import pickle
with open('saved_data/testset.bin', 'rb') as f:
  testset = pickle.load(f)

In [215]:
test_questions = df["question"].values.tolist()
test_groundtruths = df["ground_truth"].values.tolist()

In [217]:
# A helper function for evaluating each pipeline
from tqdm.auto import tqdm
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

retrieval_metrics = [
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness, # added for extra information
]

def evaluate_retrieval_chain(retrieval_chain, test_questions, test_groundtruths, metrics=None):
    if metrics is None:
        metrics = retrieval_metrics
    answers = []
    contexts = []
    for question in tqdm(test_questions):
      response = retrieval_chain.invoke({"question" : question})
      answers.append(response["response"].content)
      contexts.append([context.page_content for context in response["context"]])
        
    response_dataset = Dataset.from_dict({
        "question" : test_questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : test_groundtruths
    })
    results = evaluate(response_dataset, metrics)
    return results

In [218]:
# Some simple utilities for saving results
import inspect
import pickle

def get_variable_name(variable):
    frame = inspect.currentframe().f_back
    for name, val in frame.f_locals.items():
        if val is variable:
            return name

def save_results(results, prefix='results'):
    with open(f"saved_data/{prefix}.bin", 'wb') as f:
        pickle.dump(results, f)

In [219]:

results_naive = evaluate_retrieval_chain(naive_retrieval_chain, test_questions, test_groundtruths)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

In [220]:
save_results(results_naive, get_variable_name(results_naive))

In [221]:
print(results_naive)

{'faithfulness': 0.8475, 'context_recall': 0.9598, 'context_precision': 0.7950, 'answer_correctness': 0.7210}


In [222]:
results_bm25 = evaluate_retrieval_chain(bm25_retrieval_chain, test_questions, test_groundtruths)
save_results(results_bm25, get_variable_name(results_bm25))
print(results_bm25)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

No statements were generated from the answer.


{'faithfulness': 0.7001, 'context_recall': 0.8008, 'context_precision': 0.6625, 'answer_correctness': 0.7286}


In [223]:
results_multiquery = evaluate_retrieval_chain(multi_query_retrieval_chain, test_questions, test_groundtruths)
save_results(results_multiquery, get_variable_name(results_multiquery))
print(results_multiquery)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

{'faithfulness': 0.8731, 'context_recall': 0.9138, 'context_precision': 0.7365, 'answer_correctness': 0.7030}


In [224]:
results_parent_document = evaluate_retrieval_chain(parent_document_retrieval_chain, test_questions, test_groundtruths)
save_results(results_parent_document, get_variable_name(results_parent_document))
print(results_parent_document)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

{'faithfulness': 0.8040, 'context_recall': 0.7854, 'context_precision': 0.8525, 'answer_correctness': 0.7211}


In [225]:
results_compression = evaluate_retrieval_chain(contextual_compression_retrieval_chain, test_questions, test_groundtruths)
save_results(results_compression, get_variable_name(results_compression))
print(results_compression)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

{'faithfulness': 0.8692, 'context_recall': 0.9425, 'context_precision': 0.8908, 'answer_correctness': 0.7194}


In [226]:
results_ensemble = evaluate_retrieval_chain(ensemble_retrieval_chain, test_questions, test_groundtruths)
save_results(results_ensemble, get_variable_name(results_ensemble))
print(results_ensemble)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

{'faithfulness': 0.8611, 'context_recall': 0.9808, 'context_precision': 0.7449, 'answer_correctness': 0.7397}


**The results for semantic chunker below are not to be considered for the analysis because they are using semantically chunked documents and a naive retriever setting up a bit of an apples to orange comparison. I've left it here just to check the correctness score.**

In [227]:
# These results can be ignored (as instructed) because they are more a function of the semantically
# chunked documents than any retriever. It is simply using the naive retriever.
results_semantic = evaluate_retrieval_chain(semantic_retrieval_chain, test_questions, test_groundtruths)
save_results(results_semantic, get_variable_name(results_semantic))
print(results_semantic)

  0%|          | 0/87 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/348 [00:00<?, ?it/s]

{'faithfulness': 0.7937, 'context_recall': 0.9330, 'context_precision': 0.7694, 'answer_correctness': 0.7143}


In [228]:
results_df = pd.DataFrame([
    dict(results_naive),
    dict(results_bm25),
    dict(results_parent_document),
    dict(results_multiquery),
    dict(results_compression),
    dict(results_semantic),
    dict(results_ensemble),
])
results_df['Name'] = ['Naive', 'BM25', 'Parent Document', 'Multiquery', 'Compression', 'Semantic', 'Ensemble']
results_df.set_index('Name')

,faithfulness,context_recall,context_precision,answer_correctness
Name,,,,
Naive,0.847478,0.959770,0.795049,0.721005
BM25,0.700058,0.800766,0.662516,0.728608
Parent Document,0.804046,0.785441,0.852490,0.721137
Multiquery,0.873053,0.913793,0.736510,0.703014
Compression,0.869169,0.942529,0.890805,0.719400
Semantic,0.793654,0.932950,0.769377,0.714292
Ensemble,0.861125,0.980843,0.744850,0.739732


# Results ranked by Context Recall

In [229]:
by_context_recall = results_df.sort_values(by='context_recall', ascending=False)
by_context_recall.loc[:, ('Name', 'context_recall')]

,Name,context_recall
6,Ensemble,0.980843
0,Naive,0.959770
4,Compression,0.942529
5,Semantic,0.932950
3,Multiquery,0.913793
1,BM25,0.800766
2,Parent Document,0.785441


# Results ranked by Context Precision

In [230]:
by_context_precision = results_df.sort_values(by='context_precision', ascending=False)
by_context_precision.loc[:, ('Name', 'context_precision')]

,Name,context_precision
4,Compression,0.890805
2,Parent Document,0.852490
0,Naive,0.795049
5,Semantic,0.769377
6,Ensemble,0.744850
3,Multiquery,0.736510
1,BM25,0.662516


# Results ranked by Faithfulness

In [231]:
by_faithfulness = results_df.sort_values(by='faithfulness', ascending=False)
by_faithfulness.loc[:, ('Name', 'faithfulness')]

,Name,faithfulness
3,Multiquery,0.873053
4,Compression,0.869169
6,Ensemble,0.861125
0,Naive,0.847478
2,Parent Document,0.804046
5,Semantic,0.793654
1,BM25,0.700058


# Results ranked by correctness (not a retrieval metric)

In [233]:
by_correctness = results_df.sort_values(by='answer_correctness', ascending=False)
by_correctness.loc[:, ('Name', 'answer_correctness')]

,Name,answer_correctness
6,Ensemble,0.739732
1,BM25,0.728608
2,Parent Document,0.721137
0,Naive,0.721005
4,Compression,0.719400
5,Semantic,0.714292
3,Multiquery,0.703014


# Analysis of results of retrieval metrics

Overall, simply looking at the metrics, Compression retriever seems to be doing well on everything except context_recall. The Compression retriever does the best job on context_precision. This makes sense because the reraking pass which does a second pass of context selection in this retriever. Ensemble Retriever does the best on context_recall (also makes sense because it is fetching contexts using each of the retrievers).

# Cost/Performance Analysis

Looking at the cost/performance numbers for each of the evaluators:

Average Naive Latency/call - 1.73s\
Average Naive Cost/call - $0.0018

Average BM25 Latency/call - 1.01s\
Average BM25 Cost/call - $0.0006

Average Contextual Compression Latency/call - 1.90s\
Average Contextual Compression Cost/call - $0.0006 (However this does NOT include Cohere costs which will drive this up)

Average Multiquery Latency/call - 3.00s\
Average Multiquery Cost/call - $0.002

Average Parent Doc Latency/call - 1.60s\
Average Parent Doc Cost/call - $0.0005

Average Ensemble Retriever Latency/call - 4.83s\
Average Ensemble Retriever Cost/call - $0.003

*Note - Ignore the below result for comparison because it is just the naive retriever with semantic chunking of docs*
Average Semantic Chunking Retriever Latency - 1.07s\
Average Semantic Chunking Retriever Cost - $0.0013

In [58]:
import pandas as pd
perfcost_df = pd.DataFrame({
    'Name': ['Naive', 'BM25', 'Compression', 'Multiquery', 'Parent Document', 'Ensemble'],
    'Latency': [1.73, 1.01, 1.90, 3.00, 1.60, 4.83],
    'Cost': [0.0018, 0.0006, 0.0006, 0.002, 0.0005, 0.003,]
})

## Analyze by cost

In [60]:
perfcost_df_by_cost = perfcost_df.sort_values(by='Cost', ascending=False)
perfcost_df_by_cost.loc[:, ('Name', 'Cost')]

,Name,Cost
5,Ensemble,0.0030
3,Multiquery,0.0020
0,Naive,0.0018
1,BM25,0.0006
2,Compression,0.0006
4,Parent Document,0.0005


Note once again that the above costs ignore the cost of calling Cohere API which can quickly add up. However, *it makes sense that Ensemble and Multiquery retrievers cost the most given they do many additional operations.* For the rest of the retrievers, they are within a margin of error for comparison.

## Analyze by latency

In [61]:
perfcost_df_by_cost = perfcost_df.sort_values(by='Latency', ascending=False)
perfcost_df_by_cost.loc[:, ('Name', 'Latency')]

,Name,Latency
5,Ensemble,4.83
3,Multiquery,3.00
2,Compression,1.90
0,Naive,1.73
4,Parent Document,1.60
1,BM25,1.01


Once again, it makes sense that Ensemble and Multiquery retrievers take the longest time.